In [28]:
import requests
from bs4 import BeautifulSoup

url = "https://thehimalayantimes.com"
html_text = requests.get(url).text

In [29]:
soup = BeautifulSoup(html_text, 'lxml')

In [30]:
articles = soup.find_all('h3',class_ ='alith_post_title' )
# get the articles 

In [31]:
x= articles[0].find('a')['href']

In [32]:
url = x
html_text = requests.get(url).text

In [33]:
text_article = BeautifulSoup(html_text,'lxml')
text =text_article.find('div',class_ = 'single-content animate-box')

In [40]:
x = text.findAll('p')[1:][3].text
x

'Similarly, the central bank has fined Himalayan Bank more than Rs 10.39 million for not maintaining the CD ratio in mid-March.'

In [37]:
x.split('.')

['Of the three banks, Kumari Bank and Himalayan Bank, which were conducting financial activities against the set rules, have been fined nearly Rs 17 million in total by the central bank',
 '']

In [51]:
# Lets take 10 articles
for article in articles[:10] :
    x = article.find('a')['href'] # Get the url
    html_text = requests.get(x).text # go to the page of the article
    text_article = BeautifulSoup(html_text,'lxml') 
    text =text_article.find('div',class_ = 'single-content animate-box') # find the paragraphs
   # y= text.findAll('p')[1:][0].text
    paragraphs = text.findAll('p')[1:]
    for pargraph in paragraphs :
        y = pargraph.text
        with open('scrapedData.txt', 'a+') as f:# write to a filejj
            #f.write(y)
            f.seek(0)
            data = f.read(100)
            if len(data)>0:
                f.write('\n')

            f.write(y)



    

     

In [135]:
with open('scrapedData.txt', 'r') as f:
    text_data_from_file = f.read()
    

In [57]:
text_data_from_file_list = text_data_from_file.split('\n') # Make list of the sentences

In [44]:
# POS TAGGING

import spacy 
nlp = spacy.load("en_core_web_sm")
doc = nlp(text_data_from_file)

In [ ]:
for token in doc:
    print(token , " | ", token.pos_ )

The  |  DET
incessant  |  ADJ
rainfall  |  NOUN
occurred  |  VERB
since  |  SCONJ
last  |  ADJ
night  |  NOUN
has  |  AUX
caused  |  VERB
inundation  |  NOUN
in  |  ADP
different  |  ADJ
parts  |  NOUN
of  |  ADP
the  |  DET
Kathmandu  |  PROPN
Valley  |  PROPN
.  |  PUNCT
The  |  DET
District  |  PROPN
Police  |  PROPN
Premises  |  VERB
Kathmandu  |  PROPN
has  |  AUX
said  |  VERB
Dhungedhara  |  PROPN
,  |  PUNCT
Gaurighat  |  PROPN
,  |  PUNCT
Guheshwori  |  PROPN
,  |  PUNCT
Kapan  |  PROPN
,  |  PUNCT
Samakhushi  |  PROPN
and  |  CCONJ
Anamnagar  |  PROPN
,  |  PUNCT
among  |  ADP
others  |  NOUN
have  |  AUX
been  |  AUX
inundated  |  VERB
with  |  ADP
downpour  |  NOUN
.  |  PUNCT
Senior  |  ADJ
Superintendent  |  PROPN
of  |  ADP
Police  |  PROPN
Dan  |  PROPN
Bahadur  |  PROPN
Karki  |  PROPN
informed  |  VERB
,  |  PUNCT
"  |  PUNCT
Heavy  |  ADJ
rainfall  |  NOUN
occurred  |  VERB
since  |  SCONJ
last  |  ADJ
night  |  NOUN
inundated  |  VERB
many  |  ADJ
places  |  NOUN
in

In [ ]:
# entities
for ent in doc.ents:
    print(ent.text , " | ",ent.label_, spacy.explain(ent.label_) )

last night  |  TIME Times smaller than a day
the Kathmandu Valley  |  LOC Non-GPE locations, mountain ranges, bodies of water
The District Police Premises Kathmandu  |  ORG Companies, agencies, institutions, etc.
Dhungedhara  |  GPE Countries, cities, states
Gaurighat  |  GPE Countries, cities, states
Guheshwori  |  GPE Countries, cities, states
Kapan  |  GPE Countries, cities, states
Samakhushi  |  GPE Countries, cities, states
Anamnagar  |  GPE Countries, cities, states
Dan Bahadur Karki  |  PERSON People, including fictional
last night  |  TIME Times smaller than a day
the Kathmandu Valley  |  LOC Non-GPE locations, mountain ranges, bodies of water
Bagmati  |  GPE Countries, cities, states
Manohara Rivers  |  PERSON People, including fictional
Budhanilakantha  |  GPE Countries, cities, states
the Kathmandu Valley  |  LOC Non-GPE locations, mountain ranges, bodies of water
The Weather Forecasting Division  |  ORG Companies, agencies, institutions, etc.
Bishnumati  |  GPE Countries, c

In [136]:
#All of our entities

Entities = []
def AddEntities(entity):
    for Entity in Entities:
        if Entity == entity:
            return
        
    Entities.append(entity)

In [137]:
from transformers import pipeline

triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')
# We need to use the tokenizer manually since we need special tokens.
relation_list = []
# Function to parse the generated text and extract the triplets
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
               # AddEntities(subject)
            elif current == 's':
                object_ += ' ' + token
               # AddEntities(object_)
            elif current == 'o':
                relation += '_' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets

for relation_text in text_data_from_file_list:
    extracted_text = triplet_extractor.tokenizer.batch_decode([triplet_extractor(relation_text, return_tensors=True, return_text=False)[0]["generated_token_ids"]])
   # print(extracted_text[0])
    extracted_triplets = extract_triplets(extracted_text[0])
    relation_list += extracted_triplets
    #print(extracted_triplets)


In [138]:
Entities

[]

In [139]:
import json
final = json.dumps(relation_list, indent=2)
print(final)

[
  {
    "head": "Kathmandu Valley",
    "type": "_instance_of",
    "tail": "inundation"
  },
  {
    "head": "Kathmandu Valley",
    "type": "_located_in_or_next_to_body_of_water",
    "tail": "Bagmati"
  },
  {
    "head": "Kathmandu Valley",
    "type": "_located_in_or_next_to_body_of_water",
    "tail": "Manohara River"
  },
  {
    "head": "Waterlogged roads and schools",
    "type": "_has_cause",
    "tail": "inundations"
  },
  {
    "head": "inundations",
    "type": "_has_effect",
    "tail": "Waterlogged roads and schools"
  },
  {
    "head": "Budhanilakantha",
    "type": "_located_in_the_administrative_territorial_entity",
    "tail": "Kathmandu Valley"
  },
  {
    "head": "Sundarijal",
    "type": "_located_in_the_administrative_territorial_entity",
    "tail": "Kathmandu Valley"
  },
  {
    "head": "Nagarkot",
    "type": "_located_in_the_administrative_territorial_entity",
    "tail": "Kathmandu Valley"
  },
  {
    "head": "Kathmandu Valley",
    "type": "_contains

In [140]:
with open("sample.json", "w") as outfile:
    outfile.write(final)

In [141]:
f = open('sample.json')

data = json.load(f)

In [142]:
for relation in data:
    AddEntities(relation['head'])
    AddEntities(relation['tail'])



In [145]:
data

[{'head': 'Kathmandu Valley', 'type': '_instance_of', 'tail': 'inundation'},
 {'head': 'Kathmandu Valley',
  'type': '_located_in_or_next_to_body_of_water',
  'tail': 'Bagmati'},
 {'head': 'Kathmandu Valley',
  'type': '_located_in_or_next_to_body_of_water',
  'tail': 'Manohara River'},
 {'head': 'Waterlogged roads and schools',
  'type': '_has_cause',
  'tail': 'inundations'},
 {'head': 'inundations',
  'type': '_has_effect',
  'tail': 'Waterlogged roads and schools'},
 {'head': 'Budhanilakantha',
  'type': '_located_in_the_administrative_territorial_entity',
  'tail': 'Kathmandu Valley'},
 {'head': 'Sundarijal',
  'type': '_located_in_the_administrative_territorial_entity',
  'tail': 'Kathmandu Valley'},
 {'head': 'Nagarkot',
  'type': '_located_in_the_administrative_territorial_entity',
  'tail': 'Kathmandu Valley'},
 {'head': 'Kathmandu Valley',
  'type': '_contains_administrative_territorial_entity',
  'tail': 'Budhanilakantha'},
 {'head': 'Kathmandu Valley',
  'type': '_contains_

In [167]:
from neo4j import GraphDatabase

URI = "neo4j+s://fb99990e.databases.neo4j.io"
AUTH = ("neo4j", "lh1LhGuDXcqbw3e4-F4DIN9iFoQLbXY3fiMotLoG9Ac")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [170]:
for name in Entities:
    records, summary, keys = driver.execute_query(
        """CREATE (:Entity{name:$namae})"""
        ,namae=name,
        database_="neo4j")


In [149]:
data

[{'head': 'Kathmandu Valley', 'type': '_instance_of', 'tail': 'inundation'},
 {'head': 'Kathmandu Valley',
  'type': '_located_in_or_next_to_body_of_water',
  'tail': 'Bagmati'},
 {'head': 'Kathmandu Valley',
  'type': '_located_in_or_next_to_body_of_water',
  'tail': 'Manohara River'},
 {'head': 'Waterlogged roads and schools',
  'type': '_has_cause',
  'tail': 'inundations'},
 {'head': 'inundations',
  'type': '_has_effect',
  'tail': 'Waterlogged roads and schools'},
 {'head': 'Budhanilakantha',
  'type': '_located_in_the_administrative_territorial_entity',
  'tail': 'Kathmandu Valley'},
 {'head': 'Sundarijal',
  'type': '_located_in_the_administrative_territorial_entity',
  'tail': 'Kathmandu Valley'},
 {'head': 'Nagarkot',
  'type': '_located_in_the_administrative_territorial_entity',
  'tail': 'Kathmandu Valley'},
 {'head': 'Kathmandu Valley',
  'type': '_contains_administrative_territorial_entity',
  'tail': 'Budhanilakantha'},
 {'head': 'Kathmandu Valley',
  'type': '_contains_

In [164]:
len(data)

128

In [171]:
# records, summary, keys = driver.execute_query(
#     """CREATE (:User{name:"@dudue"})-[:FOLLOWS]->(:User{name:"@sederew"}) """,

#     database_="neo4j")

for relation in data:
    print(relation)
    relationp = relation['type']
    records, summary, keys = driver.execute_query(
     """MATCH (s:Entity)  WHERE s.name =  $subject
      MATCH (o:Entity)  WHERE o.name = $object 
       CREATE (s)-[:"""+relationp+"""]->(o)""",
       subject = relation['head'],
       object = relation['tail'],
  
    
     database_="neo4j")
    

{'head': 'Kathmandu Valley', 'type': '_instance_of', 'tail': 'inundation'}
{'head': 'Kathmandu Valley', 'type': '_located_in_or_next_to_body_of_water', 'tail': 'Bagmati'}
{'head': 'Kathmandu Valley', 'type': '_located_in_or_next_to_body_of_water', 'tail': 'Manohara River'}
{'head': 'Waterlogged roads and schools', 'type': '_has_cause', 'tail': 'inundations'}
{'head': 'inundations', 'type': '_has_effect', 'tail': 'Waterlogged roads and schools'}
{'head': 'Budhanilakantha', 'type': '_located_in_the_administrative_territorial_entity', 'tail': 'Kathmandu Valley'}
{'head': 'Sundarijal', 'type': '_located_in_the_administrative_territorial_entity', 'tail': 'Kathmandu Valley'}
{'head': 'Nagarkot', 'type': '_located_in_the_administrative_territorial_entity', 'tail': 'Kathmandu Valley'}
{'head': 'Kathmandu Valley', 'type': '_contains_administrative_territorial_entity', 'tail': 'Budhanilakantha'}
{'head': 'Kathmandu Valley', 'type': '_contains_administrative_territorial_entity', 'tail': 'Sundarij

In [83]:
driver.execute_query(
    "CREATE CONSTRAINT ON (user:User) ASSERT user.name IS UNIQUE" 
)

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid constraint syntax, ON and ASSERT should not be used. Replace ON with FOR and ASSERT with REQUIRE. (line 1, column 1 (offset: 0))
"CREATE CONSTRAINT ON (user:User) ASSERT user.name IS UNIQUE"
 ^}